In [44]:
from PIL import Image
import random
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot
from IPython.display import display, HTML, Image as image

# Funkcja kodująca wiadomość w obrazie

In [25]:
def encoder(image_location):
    
    image = Image.open(image_location)
    total=image.height*image.width*3/7    
    print("W wybranym zdjeciu mozna zakodowac " + str(round(total)) + " znakow, czyli ok. " + str(round(total/1800)) + " stron.")
    
    message = input('Podaj wiadomosc do zakodowania (bez polskich znakow): ')
    
    # Przekodowanie wiadomosci na strumien bitow
    
    bitstream = ""
    for ch in message:
        char = format(ord(ch), '07b')
        bitstream += char 
    bitstream += "0000000"
    for n in range(image.height*image.width*3):
        bitstream += str(random.randint(0, 1))
        
    bit = iter(bitstream)
    
    # Zamiana najmniej znaczacych bitow w pikselach
    
    encoded_image = Image.new("RGB", image.size)
    pixels = encoded_image.load()
    
    for h in range(image.height):
        for w in range(image.width):

            newred = int(bin(image.getpixel((w,h))[0])[:-1] + next(bit),2)
            newgreen = int(bin(image.getpixel((w,h))[1])[:-1] + next(bit),2)         
            newblue = int(bin(image.getpixel((w,h))[2])[:-1] + next(bit),2)
            
            pixels[w,h] = (newred, newgreen, newblue)
    
    encoded_image.save("encoded_image.png")



### Wywołanie funkcji

In [26]:
encoder("example.png")

W wybranym zdjeciu mozna zakodowac 189333 znakow, czyli ok. 105 stron.
Podaj wiadomosc do zakodowania (bez polskich znakow): Kryptografia stosowana 2018


### Porównanie zdjęcia oryginalnego (po lewej) i z ukrytą wiadomością (po prawej).

In [27]:
display(HTML("<table><tr><td><img src='example.png'></td><td><img src='encoded_image.png'></td></tr></table>"))

,


# Funkcja odczytująca wiadomość z obrazu

In [29]:
def decoder(encoded_image_location):
    
    encoded_image = Image.open(encoded_image_location)
    
    #ekstrakcja najmniej znaczacego bitu z kanalow RGB kolejnych pikseli
    
    secret_letter = ""
    secret_message = ""
    for h in range(encoded_image.height):
        for w in range(encoded_image.width):
            for n in range(3):
                secret_bit = bin(encoded_image.getpixel((w,h))[n])[-1]
                secret_letter += secret_bit
                if len(secret_letter)==7 and secret_letter!="0000000":
                    ch = chr(int(secret_letter,2))
                    secret_message += ch
                    #print(ch)
                    secret_letter = ""
                if secret_letter == "0000000":
                    return print("Sekretna wiadomosc to: " + secret_message)

### Wywołanie funkcji

In [31]:
decoder("encoded_image.png")

Sekretna wiadomosc to: Kryptografia stosowana 2018


# Porównanie zmiany koloru losowego piksela

In [43]:
image = Image.open('example.png')
encoded_image=Image.open('encoded_image.png') 

init_notebook_mode(connected=True)

before=0
after=0
while before==after:
    a = random.randint(0, 255)
    b = random.randint(0, 255)
    before = image.getpixel((a,b))[0:3]
    after = encoded_image.getpixel((a,b))
    col_b = '#%02x%02x%02x' % before
    col_a = '#%02x%02x%02x' % after

vals = [['Oryginalne zdjecie', 'Zakodowane zdjecie'], [str(before), str(after)], ["",""]]
cols = ["white","white",col_b,"white","white",col_a]

trace = go.Table(
        header=dict(values=['<b>Obraz</b>', '<b>RGB</b>', '<b>Kolor</b>']),
        cells=dict(values=vals, fill = dict(color=cols), height = 30)
        )

layout = dict(width=800, height=300, autosize=False,
              title='Zmiana w kolorze losowego piksela po zakodowaniu')
data = [trace]
fig = dict(data=data, layout=layout)
#offline.plot(fig, image='png')
iplot(fig)